In [31]:
import math
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random


Dmin = math.log(278) # lower bound for diffusivity
Dmax = math.log(400000) # upper bound for diffusivity
psi = 0 # phase

def solution(x, t, A, omega, D):
  """This is the function which gives back the hydrolic head
  omega: frequency
  A: amplitude
  x: distance
  t: time"""
  sinus = math.sin(-x*math.sqrt(omega/(2*D))+ omega*t + psi)
  ret = A*math.exp(-x*math.sqrt(omega/(2*D)))*sinus
  return ret

In [32]:
dist = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days

# Initialize empty return list
result = [[ [None for _ in range(100) ] for _ in range(3)] for _ in range(3)]

for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
    for w in range(100):
      A = math.exp(2*random.random() - 1) # sample amplitude from exponential of uniform distribution
      omega = (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 # sample frequency from uniform distribution
      D = math.exp((Dmax - Dmin)*random.random() + Dmin)

      result[i][j][w] = solution(x, t, A, omega, D)

In [33]:
fig = make_subplots(rows=3, cols=3, subplot_titles=('X = 1m, t = 7d','X = 10m, t = 7d', 'X = 100m, t = 7d','X = 1m, t = 30d', 'X = 10m, t = 30d','X = 100m, t = 30d', 'X = 1m, t = 180d','X = 10m, t = 180d', 'X = 100m, t = 180d')) # Plot histograms

fig.add_trace(go.Histogram(x=result[0][0]), row=1, col=1)
fig.add_trace(go.Histogram(x=result[0][1]), row=1, col=2)
fig.add_trace(go.Histogram(x=result[0][2]), row=1, col=3)

fig.add_trace(go.Histogram(x=result[1][0]), row=2, col=1)
fig.add_trace(go.Histogram(x=result[1][1]), row=2, col=2)
fig.add_trace(go.Histogram(x=result[1][2]), row=2, col=3)

fig.add_trace(go.Histogram(x=result[2][0]), row=3, col=1)
fig.add_trace(go.Histogram(x=result[2][1]), row=3, col=2)
fig.add_trace(go.Histogram(x=result[2][2]), row=3, col=3)

fig.update_layout(showlegend=False)
# Change y axis range

In [14]:
# Create figure
fig = go.Figure()

# Fixed parameters
D = 4
A = 3
psi = 1

# Add traces one for each time step
for step in np.arange(0, 10, 0.2):
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            name="𝜈 = " + str(step),
            x=np.arange(0, 10, 0.1),
            y=[solution(x, t, A, omega, D) for d in np.arange(0, 10, 0.1)]))

fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Timestep: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()


In [42]:
# Sensitivity analysis
import plotly.express as px

omega = (math.pi + 2*math.pi/7)/2
D = (math.exp(Dmin)+math.exp(Dmax)/2)


A_range = list(np.linspace(math.exp(-1), math.exp(1), 100))
func_value = [solution(x, t, A, omega, D) for A in A_range]

dx = A_range[1]-A_range[0]

dydx = np.gradient(func_value, dx)

fig = px.line(x=A_range, y=func_value, title='Derivative of hydraulic head w.r.t. A')

fig.show()

In [41]:
A = (math.exp(-1)+ math.exp(1))/2
D = (math.exp(Dmin)+math.exp(Dmax)/2)

omega_range = list(np.linspace(2*math.pi/7, math.pi, 100))
func_value = [solution(x, t, A, omega, D) for omega in omega_range]

dx = omega_range[1]-omega_range[0]

dydx = np.gradient(func_value, dx)

fig = px.line(x=omega_range, y=func_value, title='Derivative of hydraulic head w.r.t. omega')


fig.show()


In [40]:
A = (math.exp(-1)+ math.exp(1))/2
omega = (math.pi + 2*math.pi/7)/2

D_range = list(np.linspace(math.exp(Dmin), math.exp(Dmax), 100))
func_value = [solution(x, t, A, omega, D) for D in D_range]

dx = D_range[1]-D_range[0]

dydx = np.gradient(func_value, dx)

fig = px.line(x=D_range, y=func_value, title='Derivative of hydraulic head w.r.t. D')


fig.show()